In [1]:
import google.generativeai as genai
import pandas as pd
import time

In [2]:
# Necesitaras crear un archivo GEMINI_API_KEYS.txt con las claves de Gemini que tengas
with open('GEMINI_API_KEY.txt') as f:
    genai.configure(api_key=f.readline())

In [ ]:
MIN_LENGTH = 300
MAX_LENGTH = 10000000 # Esencialmente infinito
LIMIT = 1495
START_INDEX = 1219

In [3]:
df = pd.read_excel('DESCRIPCIONES_TODAS.xlsx')
df = df.dropna(subset='Código de la especie (id taxon)')
df['Código de la especie (id taxon)'] = df['Código de la especie (id taxon)'].astype(int)
df = df[~df.duplicated(subset='Código de la especie (id taxon)', keep=False)]
df = df[(df['Descripción'].str.len() < MAX_LENGTH) & (df['Descripción'].str.len() >= MIN_LENGTH)].copy().reset_index(drop=True)
df

,Nombres comunes,Nombre aceptado,Fuente,Grupo taxonómico,Descripción,Código de la especie (id taxon)
0,Acentor alpino,Prunella collaris,SEO BirdLife,Aves,El acentor alpino es un pájaro robusto y compa...,11054
1,Acentor común,Prunella modularis,SEO BirdLife,Aves,El acentor común es un pájaro del tamaño de un...,11761
2,Agachadiza chica,Lymnocryptes minimus,SEO BirdLife,Aves,"La agachadiza chica es una limícola pequeña, d...",11738
3,Agachadiza común,Gallinago gallinago,SEO BirdLife,Aves,La agachadiza común es una limícola de mediano...,11337
4,Abejaruco persa,Merops persicus,SEO BirdLife,Aves,"El abejaruco persa, de la talla del abejaruco ...",16369
...,...,...,...,...,...,...
2244,Culebra bastarda,Malpolon monspessulanus,VertebradosIbericos - Reptiles,Reptiles,"Cabeza alta, estrecha, con hocico agudo. Escam...",11772
2245,Culebrilla de las macetas,Ramphotyphlops braminus,VertebradosIbericos - Reptiles,Reptiles,Pequeño ofidio perteneciente a la familia Typh...,36635
2246,Víbora áspid,Vipera aspis,VertebradosIbericos - Reptiles,Reptiles,Se trata de una serpiente de mediano tamaño (m...,10772
2247,Víbora hocicuda,Vipera latastei Boscá,VertebradosIbericos - Reptiles,Reptiles,"Se caracteriza por su cuerpo grueso y corto, c...",11567


In [5]:
model = genai.GenerativeModel('gemini-2.0-flash')
df['Descripción Gemini 1'] = ''
df['Descripción Gemini 2'] = ''
df['Descripción Gemini 3'] = ''
for i, row in df.iterrows():
    if i < START_INDEX:
        continue
    if i >= START_INDEX + LIMIT:
        break
    if (i+1) % 15 == 0:
        print(f'Starting prompt {i+1}...', end='\r')
        time.sleep(60)
    messages = [
        {'role': 'user',
        'parts': ["""Quiero fabricar descripciones de especies que una persona común sin conocimientos técnicos pueda usar para identificar las especies en la naturaleza.
        Convierte la descripción que te voy a dar a continuación en una descripción más sencilla, que sea de aproximadamente 100 palabras.
        No incluyas información que no esté en la descripción original, y devuelve solamente la descripción simplificada.
        Puedes omitir información que no sea tan relevante para la identificación de una persona común (que no sabe nombres científicos) y cambiar la estructura.
        También puedes unir frases si así se consigue una lectura más fluida y sin tantas oraciones separadas."""]},
        {'role': 'model',
        'parts': ["Entendido."]},
        {'role': 'user',
        'parts': [row['Descripción']]}
    ]
    response = model.generate_content(messages, generation_config=genai.types.GenerationConfig(candidate_count=3))
    descriptions = [c.content.parts[0].text.strip() for c in response.candidates]
    df.loc[i, 'Descripción Gemini 1'] = descriptions[0]
    df.loc[i, 'Descripción Gemini 2'] = descriptions[1]
    df.loc[i, 'Descripción Gemini 3'] = descriptions[2]
print(f'Finished all {i - START_INDEX} prompts')
df.rename(columns={'Descripción': 'Descripción Original'}).to_excel(f'Descripciones_Gemini_300+_{i}.xlsx', index=False)

KeyboardInterrupt: 